In [1]:
import pandas as pd

## 此处为断点重跑的2份数据

In [2]:
# dt = pd.concat([pd.read_csv("../data/follows_deep3_self2.csv"),pd.read_excel("../data/follows_deep3_self.xlsx")])

In [3]:
dt = pd.read_csv("../data/user2up.csv")

In [4]:
dt.rename(columns={"mid":"up","from":"user"},inplace=True)

In [5]:
dt.up = dt.up.astype(int)

In [6]:
dt = dt[:10000]

In [7]:
dt.columns

Index(['up', 'attribute', 'mtime', 'tag', 'special', 'contract_info', 'uname',
       'face', 'sign', 'official_verify', 'vip', 'user'],
      dtype='object')

In [8]:
len(dt["user"].unique()),len(dt["up"].unique())

(86, 6734)

In [9]:
dt_sim = dt[["user","up"]]

In [10]:
matrix_user2up = dt_sim.groupby("user").agg(list)

In [11]:
matrix_user2up

,up
user,
1,"[335115, 2, 12, 9099524, 867152]"
2,"[1868902080, 623512391, 43855, 520811440, 3941..."
4,"[9617619, 7584632, 2026561407, 22721144, 48163..."
9,"[2223856, 698750420, 645456648, 282994, 187100..."
12,"[7788379, 313530204, 14110780, 481774557, 2054..."
...,...
477655087,"[489004577, 235555226, 928123, 326499679, 3344..."
485703766,"[31532234, 471130755, 5633240, 661904317, 3852..."
495419424,"[946974, 686150641, 3530725, 11336264, 7584632..."


In [12]:
len(dt.up.unique())

6734

## 用户-UP主倒排表 

In [13]:
matrix_up_reverse = dt_sim.groupby("up").agg(list)

In [14]:
matrix_up_reverse

,user
up,
1,[44974083]
2,"[44974083, 12, 1, 93855752, 18, 19, 13705279, ..."
4,[9]
9,"[12, 2, 4, 18, 20, 19, 75, 106, 109, 848008]"
12,"[1, 2, 4, 18, 20, 19, 38965308, 75, 3743849, 1..."
...,...
2014688638,[610390]
2016841403,[28278842]
2026561407,"[9, 2, 4, 19, 50212909, 3743849, 109, 684169, ..."


In [15]:
users = dt["user"].unique()

In [16]:
ups = dt["up"].unique()

## 用户-用户相交矩阵(重复关注UP主累计)

In [17]:
from itertools import combinations,chain

In [18]:
from tqdm import tqdm

In [19]:
matrix_user2user = pd.DataFrame(index=users,columns=users)
matrix_user2user.loc[:,:] = 0
for up,user in tqdm(zip(matrix_up_reverse.index,matrix_up_reverse.user)):
    for i,j in combinations(user,2):
        matrix_user2user.loc[i,j] += 1
        matrix_user2user.loc[j,i] += 1

6734it [00:03, 1788.13it/s]


In [20]:
users[:20]

array([    81922,  44974083,         9,        12,  31113228,  77021198,
         6701071,        16,  37781521,   5242903,         1,         2,
       369750017,         4,  93855752, 382820360,        18,   2371603,
              20,    745493], dtype=int64)

In [21]:
matrix_user2up["up_count"] = matrix_user2up["up"].apply(len)

In [22]:
matrix_user2up

,up,up_count
user,,
1,"[335115, 2, 12, 9099524, 867152]",5
2,"[1868902080, 623512391, 43855, 520811440, 3941...",224
4,"[9617619, 7584632, 2026561407, 22721144, 48163...",66
9,"[2223856, 698750420, 645456648, 282994, 187100...",250
12,"[7788379, 313530204, 14110780, 481774557, 2054...",214
...,...,...
477655087,"[489004577, 235555226, 928123, 326499679, 3344...",12
485703766,"[31532234, 471130755, 5633240, 661904317, 3852...",44
495419424,"[946974, 686150641, 3530725, 11336264, 7584632...",8


In [23]:
users[:50]

array([    81922,  44974083,         9,        12,  31113228,  77021198,
         6701071,        16,  37781521,   5242903,         1,         2,
       369750017,         4,  93855752, 382820360,        18,   2371603,
              20,    745493,        19,   4034583, 495419424,    606243,
        12824612,  97177641,  30949419,   2666541,  50212909, 477655087,
         7807024,  40857648,  33353778,  28278842,    872507,  14110780,
        38965308,  49246269,  13705279,    274491, 346165316,        75,
        20951119, 144900177,   1318997,    610390, 485703766,  35225688,
          782425,    913497], dtype=int64)

## 用户间相似度(jaccard)

In [24]:
def jaccard(user2up,user2user,user_a,user_b):
    return user2user[user_a][user_b]/len(set(user2up["up"][user_a])|set(user2up["up"][user_b])) if user2user.loc[user_a,user_b] else 0

In [25]:
jaccard(matrix_user2up,matrix_user2user,44974083,13705279)

0.014285714285714285

## 用户-用户相交矩阵(相似度) 

In [26]:
sim_user2user = pd.DataFrame(index=users,columns=users)

In [27]:
sim_user2user.loc[:,:] = 0

In [28]:
for user_a,user_b in tqdm(combinations(users,2)):
    sim = jaccard(matrix_user2up,matrix_user2user,user_a,user_b)
    sim_user2user[user_a][user_b] = sim
    sim_user2user[user_b][user_a] = sim

3655it [00:00, 6169.64it/s]


In [29]:
sim_user2user

,81922,44974083,9,12,31113228,77021198,6701071,16,37781521,5242903,...,42619059,1847476,442549,1724598,281198778,4407484,391876799,23281857,23326926,53456
81922,0,0.0229885,0.00677966,0.0155642,0.0192308,0,0.0295567,0,0.0136519,0.00813008,...,0,0.0348432,0.0167364,0.0263158,0,0.0144231,0,0.0175439,0.0242215,0.0111732
44974083,0.0229885,0,0.0187166,0.0267857,0.00680272,0.00549451,0.0501792,0.00714286,0.0132979,0,...,0.00735294,0.00793651,0.021875,0.00621118,0.00275482,0.0137457,0,0,0.0106383,0.0192308
9,0.00677966,0.0187166,0,0.165829,0.0297767,0.00332226,0.0377834,0,0.0141988,0,...,0,0.010101,0.0045045,0.00716846,0.00207469,0.0247525,0.00395257,0,0.026749,0.0105263
12,0.0155642,0.0267857,0.165829,0,0.0498615,0.00757576,0.0532213,0,0.0153173,0,...,0,0.0265487,0.0199005,0.0208333,0,0.0356164,0.00925926,0.00446429,0.0311804,0.00288184
31113228,0.0192308,0.00680272,0.0297767,0.0498615,0,0,0.021875,0,0.00973236,0,...,0,0.0121951,0.0169014,0.0103093,0.00505051,0.00611621,0,0,0.0222222,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4407484,0.0144231,0.0137457,0.0247525,0.0356164,0.00611621,0,0.0550162,0,0.0272953,0.00416667,...,0,0.0247525,0.00840336,0.00515464,0,0,0.0120482,0.0174419,0.0273632,0.003367
391876799,0,0,0.00395257,0.00925926,0,0,0.0121951,0,0.00395257,0,...,0,0,0,0,0,0.0120482,0,0,0.00796813,0
23281857,0.0175439,0,0,0.00446429,0,0,0.00581395,0,0.00772201,0,...,0,0,0,0,0,0.0174419,0,0,0.003861,0
23326926,0.0242215,0.0106383,0.026749,0.0311804,0.0222222,0.00668896,0.0275,0,0.0800866,0,...,0.00790514,0.0121704,0.0113636,0.0181818,0.002079,0.0273632,0.00796813,0.003861,0,0.00524934


## 用户-对UP主的兴趣度

In [30]:
ups

array([326499679, 316568752,   9617619, ...,  16423300, 344194038,
          472346])

In [31]:
list(ups[789:800])

[7633151,
 2394319,
 266876176,
 33881729,
 1848018,
 382651856,
 346165316,
 488423046,
 813818,
 486969127,
 14269054]

In [33]:
# sim_user2user[sim_user2user[50952087]>0][50952087]

In [34]:
cache = {}

In [35]:
class Interest(object):
    def __init__(self,up_reverse,sim_user2user):
        self.up_reverse = up_reverse
        self.sim_user2user = sim_user2user
        self.cache = {}
    def interest_user2up(self,user,up):
        cache_value = self.cache.get((user,up))
        if cache_value:
            return cache_value
        
        interest = 0
        for other_user in self.up_reverse.loc[up,"user"]:
            interest += self.sim_user2user.loc[user,other_user]
        self.cache[(user,up)] = interest
        return interest

In [36]:
ups[123:134]

array([413535812, 335505768, 135980509, 278093107, 477655087, 472547659,
         3589472,  28672699, 451066708,  62594335, 403928672])

In [37]:
interests = Interest(matrix_up_reverse,sim_user2user) 

In [39]:
# interests.interest_user2up(50952087,318240083)

## 用户-K个最相似用户-对UP主关注度-关注度排序-推荐topN个UP主

In [43]:
# sim_user2user[351502018].sort_values(ascending=False)

In [44]:
sim_user2user_rank = sim_user2user.rank(ascending=False,method="max")

In [45]:
sim_user2user_rank = sim_user2user_rank.astype(int)

In [46]:
sim_user2user_rank["top3_user"] = sim_user2user_rank.apply(lambda x: list(x[x<3].index),axis=1)

In [47]:
sim_user2user_rank["top3_user_len"] = sim_user2user_rank["top3_user"].apply(len)

In [48]:
sim_user2user_rank

,81922,44974083,9,12,31113228,77021198,6701071,16,37781521,5242903,...,442549,1724598,281198778,4407484,391876799,23281857,23326926,53456,top3_user,top3_user_len
81922,86,17,53,43,25,86,22,86,41,8,...,17,19,86,27,86,5,26,13,[],0
44974083,15,86,29,27,47,38,4,8,42,86,...,5,53,13,30,86,86,51,3,"[93855752, 4034583, 326664284, 13148307]",4
9,57,27,86,1,12,49,10,86,38,86,...,54,52,24,10,19,86,23,17,"[12, 274541, 848008, 11313303]",4
12,31,11,1,86,5,26,2,86,33,86,...,7,25,86,3,8,19,15,46,"[9, 6701071, 2, 18, 109, 274541, 848008, 11313...",8
31113228,21,62,16,14,86,86,27,86,47,86,...,16,44,3,51,86,86,29,86,"[13705279, 315487, 13725848]",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4407484,35,34,21,19,49,86,1,86,18,22,...,41,55,86,86,6,6,21,45,[6701071],1
391876799,86,86,57,53,86,86,52,86,59,86,...,86,86,86,35,86,86,58,86,[745493],1
23281857,24,86,86,61,86,86,56,86,51,86,...,86,86,86,17,86,86,65,86,[],0
23326926,11,38,18,22,20,32,23,86,2,86,...,35,30,20,7,9,23,86,37,"[37781521, 7807024, 28278842, 346165316, 3743849]",5


In [49]:
sim_user2user_rank["top3_user_all_ups"] = sim_user2user_rank["top3_user"].apply(lambda x: list(chain(*matrix_user2up.loc[x,"up"])))
sim_user2user_rank["top3_user_all_ups_len"] = sim_user2user_rank["top3_user_all_ups"].apply(len)

In [50]:
sim_user2user_rank

,81922,44974083,9,12,31113228,77021198,6701071,16,37781521,5242903,...,281198778,4407484,391876799,23281857,23326926,53456,top3_user,top3_user_len,top3_user_all_ups,top3_user_all_ups_len
81922,86,17,53,43,25,86,22,86,41,8,...,86,27,86,5,26,13,[],0,[],0
44974083,15,86,29,27,47,38,4,8,42,86,...,13,30,86,86,51,3,"[93855752, 4034583, 326664284, 13148307]",4,"[693340454, 233114659, 22135876, 22723402, 646...",174
9,57,27,86,1,12,49,10,86,38,86,...,24,10,19,86,23,17,"[12, 274541, 848008, 11313303]",4,"[7788379, 313530204, 14110780, 481774557, 2054...",798
12,31,11,1,86,5,26,2,86,33,86,...,86,3,8,19,15,46,"[9, 6701071, 2, 18, 109, 274541, 848008, 11313...",8,"[2223856, 698750420, 645456648, 282994, 187100...",1510
31113228,21,62,16,14,86,86,27,86,47,86,...,3,51,86,86,29,86,"[13705279, 315487, 13725848]",3,"[147166910, 641975239, 10330740, 517327498, 59...",462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4407484,35,34,21,19,49,86,1,86,18,22,...,86,86,6,6,21,45,[6701071],1,"[1999942692, 1373037, 476704454, 630320407, 38...",162
391876799,86,86,57,53,86,86,52,86,59,86,...,86,35,86,86,58,86,[745493],1,"[617459493, 674622242, 674421433, 18149131, 11...",32
23281857,24,86,86,61,86,86,56,86,51,86,...,86,17,86,86,65,86,[],0,[],0
23326926,11,38,18,22,20,32,23,86,2,86,...,20,7,9,23,86,37,"[37781521, 7807024, 28278842, 346165316, 3743849]",5,"[5970160, 130636947, 883974, 20124052, 1449001...",1107


In [51]:
sim_user2user_rank["interests"] = sim_user2user_rank.apply(lambda x:{up:interests.interest_user2up(x.name,up) for up in x.top3_user_all_ups},axis=1)

In [52]:
sim_user2user_rank["top10_interests"] = sim_user2user_rank.apply(lambda x: sorted(x.interests.items(),key=lambda item:item[1],reverse=True)[:10],axis=1)

In [53]:
sim_user2user_rank["top10_up"] = sim_user2user_rank["top10_interests"].apply(lambda x: [i[0] for i in x])

In [55]:
sim_user2user_rank

,81922,44974083,9,12,31113228,77021198,6701071,16,37781521,5242903,...,23281857,23326926,53456,top3_user,top3_user_len,top3_user_all_ups,top3_user_all_ups_len,interests,top10_interests,top10_up
81922,86,17,53,43,25,86,22,86,41,8,...,5,26,13,[],0,[],0,{},[],[]
44974083,15,86,29,27,47,38,4,8,42,86,...,86,51,3,"[93855752, 4034583, 326664284, 13148307]",4,"[693340454, 233114659, 22135876, 22723402, 646...",174,"{693340454: 0.06315789473684211, 233114659: 0....","[(208259, 0.4982066438712336), (401742377, 0.3...","[208259, 401742377, 353840826, 43536, 27144252..."
9,57,27,86,1,12,49,10,86,38,86,...,86,23,17,"[12, 274541, 848008, 11313303]",4,"[7788379, 313530204, 14110780, 481774557, 2054...",798,"{7788379: 0.3238674473606232, 313530204: 0.165...","[(208259, 0.7784523549939334), (9617619, 0.672...","[208259, 9617619, 2306780, 777536, 174501086, ..."
12,31,11,1,86,5,26,2,86,33,86,...,19,15,46,"[9, 6701071, 2, 18, 109, 274541, 848008, 11313...",8,"[2223856, 698750420, 645456648, 282994, 187100...",1510,"{2223856: 0.527298974185039, 698750420: 0.1658...","[(208259, 1.0095969298576306), (174501086, 0.9...","[208259, 174501086, 326499679, 37090048, 96176..."
31113228,21,62,16,14,86,86,27,86,47,86,...,86,29,86,"[13705279, 315487, 13725848]",3,"[147166910, 641975239, 10330740, 517327498, 59...",462,"{147166910: 0.15876645876645878, 641975239: 0....","[(9617619, 0.6752241531728922), (777536, 0.638...","[9617619, 777536, 208259, 423895, 37090048, 16..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4407484,35,34,21,19,49,86,1,86,18,22,...,6,21,45,[6701071],1,"[1999942692, 1373037, 476704454, 630320407, 38...",162,"{1999942692: 0.07256004087889627, 1373037: 0.0...","[(208259, 0.470588096314095), (326499679, 0.46...","[208259, 326499679, 174501086, 2306780, 489412..."
391876799,86,86,57,53,86,86,52,86,59,86,...,86,58,86,[745493],1,"[617459493, 674622242, 674421433, 18149131, 11...",32,"{617459493: 0.02857142857142857, 674622242: 0....","[(434334701, 0.05848209125912693), (18149131, ...","[434334701, 18149131, 2810223, 198297, 113711,..."
23281857,24,86,86,61,86,86,56,86,51,86,...,86,65,86,[],0,[],0,{},[],[]
23326926,11,38,18,22,20,32,23,86,2,86,...,23,86,37,"[37781521, 7807024, 28278842, 346165316, 3743849]",5,"[5970160, 130636947, 883974, 20124052, 1449001...",1107,"{5970160: 0.3331915215448073, 130636947: 0.139...","[(37090048, 0.8868424504933275), (235555226, 0...","[37090048, 235555226, 517327498, 208259, 32649..."


In [57]:
sim_user2user_rank.loc[31113228,"top10_up"]

[9617619,
 777536,
 208259,
 423895,
 37090048,
 168598,
 562197,
 7552204,
 5970160,
 546195]